In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3

In [ ]:
#multi-label binarization
#splitting multiple labels and getting a list

df = pd.read_csv("/home/umar/ML-AI/datasets/plant-pathology-2021-fgvc8/train.csv")
df["labels"]=df["labels"].apply(lambda x:x.split(" ")) 

train_path='/home/umar/ML-AI/datasets/plant-pathology-2021-fgvc8/train_images'

In [ ]:
img_size=(128,128)

datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator=datagen.flow_from_dataframe(
dataframe=df[:16000],
directory=train_path,
x_col="image",
y_col="labels",
batch_size=64,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=img_size)

val_generator=datagen.flow_from_dataframe(
dataframe=df[16000:],
directory=train_path,
x_col="image",
y_col="labels",
batch_size=64,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=img_size)

print(train_generator.class_indices)

In [ ]:
model_pretrained = InceptionV3(weights='imagenet', 
                      include_top=False, 
                      input_shape=(128,128,3))

model=keras.models.Sequential()
model.add(model_pretrained)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(100,activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(6,activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
# Time based learning rate scheduling
epoch=50
learning_rate=0.01
decay_rate=learning_rate/epoch
momentum=0.8
sgd=SGD(lr=learning_rate,momentum=momentum,decay=decay_rate)

In [ ]:
# Early stopping callback
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=[tf.keras.metrics.Precision()])

In [ ]:
model_history=model.fit(train_generator,epochs=epoch,validation_data=val_generator,shuffle=True, callbacks=[callback])

In [ ]:
# plotting the varoius metrics
pd.DataFrame(model_history.history).plot(figsize=(8,5))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
sub_path='/home/umar/ML-AI/datasets/plant-pathology-2021-fgvc8/sample_submission.csv'
sample_sub=pd.read_csv(sub_path)
sample_sub

In [ ]:
submission=sample_sub[['image']].copy()
submission

In [ ]:
test_path='/home/umar/ML-AI/datasets/plant-pathology-2021-fgvc8/test_images'

test_generator=datagen.flow_from_dataframe(
dataframe=submission,
directory=test_path,
x_col="image",
y_col=None,
batch_size=1,
seed=42,
shuffle=True,
class_mode=None,
target_size=img_size)

In [ ]:
prediction = model.predict(test_generator)
print(prediction)
prediction = prediction.tolist()

indices = []
for pred in prediction:
    temp = []
    for category in pred:
        if category>=0.35:
            temp.append(pred.index(category))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)
    
print(indices)

In [ ]:
category=train_generator.class_indices
key_list = list(category.keys())
result=[]

for i in indices:
    temp=[]
    for j in i:
        temp.append(str(key_list[j]))
    result.append(' '.join(temp))

print(result)

In [ ]:
submission['labels']=result
submission

In [ ]:
# Help taken from-https://www.kaggle.com/arnabs007/apple-leaf-diseases-with-inceptionresnetv2-keras
# It takes 16 hours to train on CPU so not suitable for submission.